In [1]:
#pip install requests
#pip install bs4
#pip install spacy
#pip install pymongo
#pip install spacy download https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.2.0/en_core_web_sm-3.2.0.tar.gz
#pip instal gensim
#pip install word2number
#pip install inflect
#pip install PyPDF2

In [ ]:
import requests
import time
from datetime import date
from datetime import datetime
from pymongo import MongoClient

dailyUrl = "https://data2.unhcr.org/population/get/timeseries?widget_id=292044&sv_id=54&population_group=5457&frequency=day&fromDate=1900-01-01"
aggregatedUrl = "https://data2.unhcr.org/population/get/sublocation?widget_id=292041&sv_id=54&population_group=5461&forcesublocation=0&fromDate=1900-01-01"

payload={}
headers = {
  'authority': 'data2.unhcr.org',
  'pragma': 'no-cache',
  'cache-control': 'no-cache',
  'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="99", "Google Chrome";v="99"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': '"Windows"',
  'upgrade-insecure-requests': '1',
  'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.84 Safari/537.36',
  'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
  'sec-fetch-site': 'none',
  'sec-fetch-mode': 'navigate',
  'sec-fetch-user': '?1',
  'sec-fetch-dest': 'document',
  'accept-language': 'it-IT,it;q=0.9,en-US;q=0.8,en;q=0.7,fr;q=0.6',
  'cookie': '_gcl_au=1.1.1224036697.1647677351; _ga=GA1.3.117673231.1647677351; _hjSessionUser_1968449=eyJpZCI6ImQzZTZhNWU4LWI2MDYtNTE0YS05NjZkLTI0MWI3MzA5ODhlZiIsImNyZWF0ZWQiOjE2NDc2NzczNTEzNDIsImV4aXN0aW5nIjp0cnVlfQ==; _hjSession_1968449=eyJpZCI6ImU2M2Q5Y2E5LWYzMjMtNDhlNi05ZjIzLWFiZDgxMTI2NDdiOSIsImNyZWF0ZWQiOjE2NDg2NzI2NTgwODMsImluU2FtcGxlIjpmYWxzZX0=; _hjIncludedInSessionSample=0; _hjIncludedInPageviewSample=1; _hjAbsoluteSessionInProgress=0; _gid=GA1.3.2000844644.1648672658'
}
DATE_FORMAT = "%Y-%m-%d"

def check_data_exist_and_update(obj, collection):
    existing_data = collection.find({'date': obj['date']})
    for record in existing_data:
        if record['value'] != obj['value']:
            print(f"{obj['date']} updated from: {record['value']} to: {obj['value']}")
            collection.update_one({'_id': record['_id']}, {'$set': {'value': obj['value']}})
        return True
    return False


def update_existing_data(data_list, collection):
    new_list = []
    for item in data_list:
        exists = check_data_exist_and_update(item, collection)
        if not exists:
            new_list.append(item)
    return new_list


def convert_str_to_datetime(date_str):
    try:
        return datetime.strptime(date_str, DATE_FORMAT)
    except ValueError:
        return None


def map_json_to_daily_object(date_obj, value):
    return {
        'date': date_obj,
        'value': value
    }


def map_json_to_aggregated_object(value, nation):
    return {
        'value': value,
        'nation': nation
    }


def connect_mongo(mongo_url, mongo_db_name):
    client = MongoClient(mongo_url)
    return client[mongo_db_name]


def get_collection(mongo_db, collection_name):
    return mongo_db[collection_name]


def insert_all_in_collection(items, collection):
    if items:
        collection.insert_many(items)


def remove_all_existing_data(collection):
    collection.drop()

# === Main execution ===
daily_data_response = requests.request("GET", daily_url, headers=headers, data=payload)
aggregated_data_response = requests.request("GET", aggregated_url, headers=headers, data=payload)

mongo_url = "mongodb://"
mongo_db_name = "kiev"
collection_refugee_daily_name = "refugeeDaily"
collection_refugee_aggregated_name = "refugeeAggregated"

mongo_db = connect_mongo(mongo_url, mongo_db_name)
collection_refugee_daily = get_collection(mongo_db, collection_refugee_daily_name)
collection_refugee_aggregated = get_collection(mongo_db, collection_refugee_aggregated_name)

# Process daily data
daily_list = []
for record in daily_data_response.json()['data']['timeseries']:
    date_obj = convert_str_to_datetime(record['data_date'])
    mapped_record = map_json_to_daily_object(date_obj, record['individuals'])
    daily_list.append(mapped_record)

daily_list_to_insert = update_existing_data(daily_list, collection_refugee_daily)
insert_all_in_collection(daily_list_to_insert, collection_refugee_daily)

# Process aggregated data
aggregated_list = []
for record in aggregated_data_response.json()['data']:
    mapped_record = map_json_to_aggregated_object(int(record['individuals']), record['geomaster_name'])
    aggregated_list.append(mapped_record)

remove_all_existing_data(collection_refugee_aggregated)
insert_all_in_collection(aggregated_list, collection_refugee_aggregated)

2022-02-24 00:00:00updated from : 82955 to :82955
2022-02-25 00:00:00updated from : 104787 to :104787
2022-02-26 00:00:00updated from : 147927 to :147927
2022-02-27 00:00:00updated from : 171579 to :171579
2022-02-28 00:00:00updated from : 163607 to :163607
2022-03-01 00:00:00updated from : 168167 to :168167
2022-03-02 00:00:00updated from : 173952 to :173952
2022-03-03 00:00:00updated from : 166382 to :166382
2022-03-04 00:00:00updated from : 171826 to :171826
2022-03-05 00:00:00updated from : 197818 to :197818
2022-03-06 00:00:00updated from : 205708 to :205708
2022-03-07 00:00:00updated from : 207322 to :207322
2022-03-08 00:00:00updated from : 184001 to :184001
2022-03-09 00:00:00updated from : 170501 to :170501
2022-03-10 00:00:00updated from : 134458 to :134458
2022-03-11 00:00:00updated from : 120435 to :120435
2022-03-12 00:00:00updated from : 115986 to :115986
2022-03-13 00:00:00updated from : 158341 to :158341
2022-03-14 00:00:00updated from : 114542 to :114542
2022-03-15 00: